In [1]:
import pandas as pd
test_df=pd.read_csv('test.csv')
train_df=pd.read_csv('train.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [2]:
test_df=pd.read_csv('test.csv')
train_df=pd.read_csv('train.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [3]:
df=pd.concat([train_df,test_df],ignore_index=True,sort=False)
df.shape

(10876, 5)

In [0]:
df=df.drop(columns=['keyword', 'location'])

In [5]:
df['text'] = df['text'].str.lower()
df.head()

,id,text,target
0,1,our deeds are the reason of this #earthquake m...,1.0
1,4,forest fire near la ronge sask. canada,1.0
2,5,all residents asked to 'shelter in place' are ...,1.0
3,6,"13,000 people receive #wildfires evacuation or...",1.0
4,7,just got sent this photo from ruby #alaska as ...,1.0


In [0]:
import re
#!pip install nltk
import nltk
#!pip install spacy
import spacy
import string

In [7]:
text=df['text']
text.head()


0    our deeds are the reason of this #earthquake m...
1               forest fire near la ronge sask. canada
2    all residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    just got sent this photo from ruby #alaska as ...
Name: text, dtype: object

In [0]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

text_clean = text.apply(lambda text: remove_punctuation(text))

In [9]:
df1=df.copy()
df1['text_clean']=text_clean
df1.head()

,id,text,target,text_clean
0,1,our deeds are the reason of this #earthquake m...,1.0,our deeds are the reason of this earthquake ma...
1,4,forest fire near la ronge sask. canada,1.0,forest fire near la ronge sask canada
2,5,all residents asked to 'shelter in place' are ...,1.0,all residents asked to shelter in place are be...
3,6,"13,000 people receive #wildfires evacuation or...",1.0,13000 people receive wildfires evacuation orde...
4,7,just got sent this photo from ruby #alaska as ...,1.0,just got sent this photo from ruby alaska as s...


In [0]:
test_df1=df1[df1['target'].isnull()]
test_df1=test_df1.drop(columns=['target'])
test_df1.reset_index(drop=True,inplace=True)

In [11]:
train_df1=df1[df1['target'].notna()]
train_df1.head()

,id,text,target,text_clean
0,1,our deeds are the reason of this #earthquake m...,1.0,our deeds are the reason of this earthquake ma...
1,4,forest fire near la ronge sask. canada,1.0,forest fire near la ronge sask canada
2,5,all residents asked to 'shelter in place' are ...,1.0,all residents asked to shelter in place are be...
3,6,"13,000 people receive #wildfires evacuation or...",1.0,13000 people receive wildfires evacuation orde...
4,7,just got sent this photo from ruby #alaska as ...,1.0,just got sent this photo from ruby alaska as s...


In [0]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


In [0]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_df1['text_clean'])

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(train_df1['text_clean'])
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(test_df1['text_clean'])
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [0]:
training_labels = train_df1['target']

In [0]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [20]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(training_padded, training_labels), verbose=2)

Epoch 1/30
238/238 - 1s - loss: 0.6823 - accuracy: 0.5703 - val_loss: 0.6747 - val_accuracy: 0.5703
Epoch 2/30
238/238 - 1s - loss: 0.6476 - accuracy: 0.6108 - val_loss: 0.5907 - val_accuracy: 0.7904
Epoch 3/30
238/238 - 1s - loss: 0.5217 - accuracy: 0.7896 - val_loss: 0.4519 - val_accuracy: 0.8363
Epoch 4/30
238/238 - 1s - loss: 0.4234 - accuracy: 0.8320 - val_loss: 0.3794 - val_accuracy: 0.8488
Epoch 5/30
238/238 - 1s - loss: 0.3677 - accuracy: 0.8554 - val_loss: 0.3421 - val_accuracy: 0.8692
Epoch 6/30
238/238 - 1s - loss: 0.3284 - accuracy: 0.8704 - val_loss: 0.2987 - val_accuracy: 0.8844
Epoch 7/30
238/238 - 1s - loss: 0.2974 - accuracy: 0.8844 - val_loss: 0.2725 - val_accuracy: 0.8912
Epoch 8/30
238/238 - 1s - loss: 0.2727 - accuracy: 0.8903 - val_loss: 0.2483 - val_accuracy: 0.9025
Epoch 9/30
238/238 - 1s - loss: 0.2491 - accuracy: 0.9067 - val_loss: 0.2454 - val_accuracy: 0.9094
Epoch 10/30
238/238 - 1s - loss: 0.2298 - accuracy: 0.9128 - val_loss: 0.2080 - val_accuracy: 0.9238

In [21]:
result=model.predict(testing_padded)
print(result)

[[0.7969774 ]
 [0.9443841 ]
 [0.99979985]
 ...
 [0.9987575 ]
 [0.9653015 ]
 [0.96824574]]


In [0]:
test_df1['target']=result

In [0]:
test_df1['target']=test_df1['target'].apply(lambda x:(0 if x < 0.75 else 1))

In [34]:
test_df1[test_df1['target']==1]

,id,text,text_clean,target
0,0,just happened a terrible car crash,just happened a terrible car crash,1
1,2,"heard about #earthquake is different cities, s...",heard about earthquake is different cities sta...,1
2,3,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...,1
3,9,apocalypse lighting. #spokane #wildfires,apocalypse lighting spokane wildfires,1
4,11,typhoon soudelor kills 28 in china and taiwan,typhoon soudelor kills 28 in china and taiwan,1
...,...,...,...,...
3257,10858,the death toll in a #is-suicide car bombing on...,the death toll in a issuicide car bombing on a...,1
3259,10865,storm in ri worse than last hurricane. my city...,storm in ri worse than last hurricane my citya...,1
3260,10868,green line derailment in chicago http://t.co/u...,green line derailment in chicago httptcoutbxlc...,1
3261,10874,meg issues hazardous weather outlook (hwo) htt...,meg issues hazardous weather outlook hwo httpt...,1


In [35]:
submission=test_df1.drop(columns=['text','text_clean'])
submission.reset_index(drop=True,inplace=True)
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      3263 non-null   int64
 1   target  3263 non-null   int64
dtypes: int64(2)
memory usage: 51.1 KB


In [0]:
submission.to_csv('submission.csv',index=0)